In [7]:
import torch
import numpy as np

# mの正負に応じてイジングモデルのエネルギーを計算
# Not devided by N-1
def energy(m, h, J):
    m = 2 * (m > 0.0) - 1
    return -torch.sum(h * m) - (torch.sum(J * torch.outer(m, m)) - torch.sum(torch.diagonal(J * torch.outer(m, m))))

# エネルギー計算関数 (ベクトル化)
# Not devided by N-1
def f(m, h, J, tau, alpha):
    AS = -(1 - alpha) * torch.sum(torch.log(1 - m**2))
    interaction = (torch.sum(J * torch.outer(m, m)) - torch.sum(torch.diagonal(J * torch.outer(m, m))))
    ising = -alpha * (torch.sum(h * m) + interaction)
    return AS + ising

# 勾配計算関数 (ベクトル化)
# Not devided by N-1
def update(m, h, J, tau, alpha):
    interaction_grad = -alpha * (torch.sum(J * m, dim=1) - torch.diagonal(J) * m)
    dif = interaction_grad + (1 - alpha) * m / (1 - m**2) - alpha * h
    return dif

# 勾配降下法 (改良版)
def gradient_descent(m, h, J, tau, alpha, max_iter=1000000, tol=1e-6):
    learning_rate = 0.01
    for iteration in range(max_iter):
        grad = update(m, h, J, tau, alpha)
        m = m - learning_rate * grad  # 勾配降下ステップ

        # 停滞のチェック（勾配が小さくなる場合）
        if torch.norm(grad) < tol:
            alpha = min(alpha + 0.01, 0.999)  # alpha を増加

        # alpha が 0.99 に達した場合、終了
        if alpha >= 0.999:
            break

    return energy(m, h, J)

# シード値の設定
def initialize_random_parameters(n, seed):
    torch.manual_seed(seed)
    J = torch.normal(mean=0.0, std=1.0, size=(n, n))
    return J

# 初期条件
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
n = 100
h = 0.0001
tau = torch.tensor(1000, device=device)
value_list = []

for i in range(10):
    m = torch.zeros(n, device=device)
    J = initialize_random_parameters(n, seed).to(device)
    alpha = torch.tensor(0.0, device=device)

    # 勾配降下法を実行
    value_final = gradient_descent(m, h, J, tau, alpha).cpu().item()

    # リストに追加
    value_list.append(value_final)

    # modify seed value
    seed += 1

# 最終結果を出力
print("Final mean energy:", np.mean(value_list))


Final mean energy: -609.9105499267578
